In [ ]:
!pip install ultralytics --no-deps
!pip install wandb

In [ ]:
!ls -l ../input/road-damage-dataset-potholes-cracks-and-manholes/data

In [ ]:
import os
import shutil
import yaml
import gc

from sklearn.model_selection import KFold
from ultralytics import YOLO, settings
import wandb
import torch
import multiprocessing as mp
import numpy as np

# Riproducibilità
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Impostazioni generali
PROJECT = "pothole-detector-NatureSR"
ENTITY = "pothole-detector"
NUM_FOLDS = 4  # Assicurati che sia pari
RANDOM_SEED = 0
SWEEP = None
IOU_THRESHOLD = 0.7
CONF_THRESHOLD = 0.25
INPUT_SIZE = 640
PATIENCE = 10
EPOCHS = 100

# Abilita integrazione con W&B solo nel processo principale
settings.update({"wandb": True})
wandb.login(key="<key>")

In [ ]:
# Sweep Config per bayesian search su precision media
sweep_config = {
    "method": "grid",
    "metric": {
        "name": "cv/precision_mean",
        "goal": "maximize"
    },
    "parameters": {
        "batch": {"value": 16},             # default
        "optimizer": {"value": "auto"},     # default
        "lr0": {"value": 0.01},             # default
        "lrf": {"value": 0.01},             # default
        "momentum": {"value": 0.937},       # default
        "weight_decay": {"value": 0.0005},  # default
        "dropout": {"value": 0.0},          # default
    }
}

sweep_id = SWEEP if SWEEP else wandb.sweep(sweep_config, project=PROJECT, entity=ENTITY)

# Dataset
image_dir = "/kaggle/input/road-damage-dataset-potholes-cracks-and-manholes/data/images"
label_dir = "/kaggle/input/road-damage-dataset-potholes-cracks-and-manholes/data/labels"
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]

# 🔀 Crea i fold e salva gli YAML
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=RANDOM_SEED)
fold_paths = []

for fold_idx, (train_idx, val_idx) in enumerate(kf.split(image_files)):
    print(f"Creazione del fold {fold_idx}...", end='')
    base_dir = f"runs/fold{fold_idx}"
    fold_paths.append(os.path.join(base_dir, "data.yaml"))
    
    # Crea directories
    for sub in ['images/train', 'images/val', 'labels/train', 'labels/val']:
        os.makedirs(os.path.join(base_dir, sub), exist_ok=True)
    
    # Copia file e conta classi per train
    class_counts = [0, 0, 0]
    for phase, indices in zip(['train', 'val'], [train_idx, val_idx]):
        for idx in indices:
            img_file = image_files[idx]
            lbl_file = img_file.rsplit('.', 1)[0] + '.txt'
            shutil.copy(os.path.join(image_dir, img_file), os.path.join(base_dir, f'images/{phase}/{img_file}'))
            
            src_lbl = os.path.join(label_dir, lbl_file)
            if os.path.exists(src_lbl):
                dst_lbl = os.path.join(base_dir, f'labels/{phase}/{lbl_file}')
                shutil.copy(src_lbl, dst_lbl)
                
                # Conta classi solo per train
                if phase == 'train':
                    with open(src_lbl, 'r') as f:
                        for line in f:
                            class_counts[int(line.split()[0])] += 1
    
    # Calcola weights dinamici
    inv_freq = [1 / max(f, 1) for f in class_counts]  # evita divisione per 0
    somma = sum(inv_freq)
    weights = [round(w / somma, 3) for w in inv_freq]
    
    # Salva YAML
    data = {
        'train': 'images/train', 'val': 'images/val', 
        'nc': 3,
        'names': ['pothole', 'crack', 'manhole'], 
        'weights': weights
    }
    with open(os.path.join(base_dir, 'data.yaml'), 'w') as f:
        yaml.dump(data, f)
    print("OK!")

print("Folds realizzati con successo!")

In [ ]:
def train_fold(fold, config_dict, run_id):
    """
    Funzione per il training di un singolo fold
    """

    # 🚫 Disabilita completamente wandb nei processi figli
    os.environ['WANDB_MODE'] = 'disabled'
    os.environ['WANDB_SILENT'] = 'true'
    settings.update({"wandb": False})

    class_names = ['pothole', 'crack', 'manhole']
    data_yaml_path = fold_paths[fold]
    device_id = fold % 2

    print(f"🚀 Fold {fold}: Training iniziato su GPU:{device_id}")

    try:
        model = YOLO("yolo11s.pt")

        # Training
        model.train(
            data=data_yaml_path,
            epochs=EPOCHS,
            batch=config_dict['batch'],
            imgsz=INPUT_SIZE,
            optimizer=config_dict['optimizer'],
            lr0=config_dict['lr0'],
            lrf=config_dict['lrf'],
            weight_decay=config_dict['weight_decay'],
            momentum=config_dict['momentum'],
            dropout=config_dict['dropout'],
            patience=PATIENCE,
            device=device_id,
            amp=True,
            seed=RANDOM_SEED,
            deterministic=True,
            project=PROJECT,
            name=f"sweep-{run_id}-fold{fold}",
            exist_ok=True,
            val=True,
            save=True,
            plots=False,
            verbose=True,
            workers=4,
        )

        # Validation
        val_results = model.val(
            data=data_yaml_path,
            iou=IOU_THRESHOLD,
            conf=CONF_THRESHOLD
        )
        metrics = val_results.box

        # Per classe
        precision = metrics.p
        recall = metrics.r
        f1 = metrics.f1
        ap50 = metrics.ap50
        ap = metrics.ap
        maps = metrics.maps

        # Medie
        mean_precision = metrics.mp
        mean_recall = metrics.mr
        mean_f1 = f1.mean()
        mean_ap50 = metrics.map50
        mean_ap75 = metrics.map75
        mean_ap = metrics.map
        fitness = metrics.fitness()

        # Logs per classe
        class_logs = {}
        for i, name in enumerate(class_names):
            class_logs.update({
                f"fold{fold}/precision_{name}": float(precision[i]),
                f"fold{fold}/recall_{name}": float(recall[i]),
                f"fold{fold}/f1_{name}": float(f1[i]),
                f"fold{fold}/ap50_{name}": float(ap50[i]),
                f"fold{fold}/ap_{name}": float(ap[i]),
                f"fold{fold}/map_{name}": float(maps[i])
            })

        # Logs medi
        class_logs.update({
            f"fold{fold}/precision_mean": float(mean_precision),
            f"fold{fold}/recall_mean": float(mean_recall),
            f"fold{fold}/f1_mean": float(mean_f1),
            f"fold{fold}/ap50_mean": float(mean_ap50),
            f"fold{fold}/ap75_mean": float(mean_ap75),
            f"fold{fold}/map_mean": float(mean_ap),
            f"fold{fold}/fitness": float(fitness)
        })

        # Salva risultati
        result_data = {
            "fold": fold,
            # "mean_precision": float(mean_precision),
            # "mean_recall": float(mean_recall),
            # "mean_f1": float(mean_f1),
            # "mean_ap50": float(mean_ap50),
            # "mean_ap75": float(mean_ap75),
            # "mean_ap": float(mean_ap),
            # "fitness": float(fitness),
            "class_logs": class_logs,
            "status": "success"
        }

        torch.save(result_data, f"fold{fold}_results.pt")
        print(f"✅ Fold {fold}: Completato")

        del model
        torch.cuda.empty_cache()
        gc.collect()

    except Exception as e:
        print(f"❌ Fold {fold}: Errore durante il training: {str(e)}")
        error_data = {
            "fold": fold,
            "status": "error",
            "error": str(e),
            "class_logs": {}
        }
        torch.save(error_data, f"fold{fold}_results.pt")
        
def sweep_train():
    """Funzione principale per il training con sweep"""

    run = wandb.init(project=PROJECT, entity=ENTITY)
    config = wandb.config
    run_id = wandb.run.id

    print(f"🎯 Avvio sweep run: {run_id}")
    print(f"📋 Config: {dict(config)}")

    config_dict = dict(config)

    fold_precisions, fold_recalls, fold_f1s = [], [], []

    for i in range(0, NUM_FOLDS, 2):
        print(f"\n🔄 Processando fold batch {i//2 + 1}/{NUM_FOLDS//2}")
        processes = []

        for fold in [i, i+1]:
            if fold < NUM_FOLDS:
                p = mp.Process(target=train_fold, args=(fold, config_dict, run_id))
                p.start()
                processes.append(p)

        for p in processes:
            p.join()

        print(f"📊 Batch {i//2 + 1} completato")

    print("\n📦 Raccolta risultati...")
    for fold in range(NUM_FOLDS):
        try:
            result = torch.load(f"fold{fold}_results.pt")
        
            if result["status"] == "success":
                fold_precisions.append(result["class_logs"][f"fold{fold}/precision_mean"])
                fold_recalls.append(result["class_logs"][f"fold{fold}/recall_mean"])
                fold_f1s.append(result["class_logs"][f"fold{fold}/f1_mean"])
        
                wandb.log(result["class_logs"])
                print(
                    f'✅ Fold {fold}: Precision = {result["class_logs"][f"fold{fold}/precision_mean"]:.4f} | '
                    f'Recall = {result["class_logs"][f"fold{fold}/recall_mean"]:.4f} | '
                    f'F1 = {result["class_logs"][f"fold{fold}/f1_mean"]:.4f}'
                )
        
                # 🔄 Upload del file fold{fold}_results.pt
                pt_path = f"fold{fold}_results.pt"
                if os.path.exists(pt_path):
                    artifact = wandb.Artifact(f"fold{fold}_results", type="model_results")
                    artifact.add_file(pt_path)
                    run.log_artifact(artifact)
                    print(f"📁 Fold {fold}: fold{fold}_results.pt caricato su wandb.")
                else:
                    print(f"⚠️ Fold {fold}: fold{fold}_results.pt non trovato.")
        
                # 🔄 Upload del results.csv come artifact
                csv_path = f"{PROJECT}/sweep-{run_id}-fold{fold}/results.csv"
                if os.path.exists(csv_path):
                    artifact = wandb.Artifact(f"fold{fold}_log", type="training_log")
                    artifact.add_file(csv_path)
                    run.log_artifact(artifact)
                    print(f"📁 Fold {fold}: results.csv caricato su wandb.")
                else:
                    print(f"⚠️ Fold {fold}: results.csv non trovato.")
            else:
                print(f"❌ Fold {fold}: Errore = {result.get('error', 'Unknown')}")

        except FileNotFoundError:
            print(f"⚠️ Fold {fold}: File risultati non trovato")
        except Exception as e:
            print(f"❌ Fold {fold}: Errore nel caricamento: {str(e)}")

    if fold_precisions:
        mean_cv_precision = sum(fold_precisions) / len(fold_precisions)
        mean_cv_recall = sum(fold_recalls) / len(fold_recalls)
        mean_cv_f1 = sum(fold_f1s) / len(fold_f1s)

        wandb.log({
            "cv/precision_mean": mean_cv_precision,
            "cv/recall_mean": mean_cv_recall,
            "cv/f1_mean": mean_cv_f1
        })

        print(f"\n📊 CV Precision: {mean_cv_precision:.4f}")
        print(f"📊 CV Recall:    {mean_cv_recall:.4f}")
        print(f"📊 CV F1 Score:  {mean_cv_f1:.4f}")
    else:
        print("❌ Nessun fold completato con successo!")
        wandb.log({
            "cv/precision_mean": 0.0,
            "cv/recall_mean": 0.0,
            "cv/f1_mean": 0.0
        })

    wandb.finish()
    print("🏁 Sweep run completato!")

In [ ]:
# Assicurati che NUM_FOLDS sia pari
if NUM_FOLDS % 2 != 0:
    print(f"⚠️ Warning: NUM_FOLDS ({NUM_FOLDS}) non è pari. Aumentando a {NUM_FOLDS + 1}")
    NUM_FOLDS += 1

# Avvia sweep
wandb.agent(sweep_id, 
            function=sweep_train, 
            count=1, 
            entity=ENTITY, 
            project=PROJECT
)